Setup code

In [81]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [82]:
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

query = f"""
Make 3 questions using this text.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogeneity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

query4 = """
What is a Mahjong game?
"""

query5 = """
For matrix A, B, C, prove that (AB)C=A(BC). Assume that all multiplication has a possible size.
"""

In [83]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

message = query4

prompt_template = PromptTemplate.from_template(
    """
You are the AI that separates instruction and context from the user input.
In this query, separate the instruction and context.
Do not answer to the query, just separate it. Also, only use given query itself, do not generate or change any words.
Instruction is saying about what AI should do.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided, then print context "".
Instruction and context must not have duplicate contents.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": message})

instr, context = [s.strip() for s in answer.split("Instruction:")[1].split("Context:")]
print(instr)
print(context)

What is a Mahjong game?
""


In [84]:
prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", determine instruction can be done only using context.
Just answer it in "Yes" or "No".
---
Context : {context}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context})
print(answer)

No


In [85]:
import os
from tavily import TavilyClient

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
add = ""

if(answer == "No"): # find context from tavily
    context = str(tavily.search(query=instr))
    add = "\nAnswer with references."

In [86]:
prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", divide instruction into 5 steps.
Do not answer or calculate for each steps, and only divide instructions to the smaller problem.
Only print 1., 2., 3., ... steps to do. Each step should be a one-liner.
Refer to the context if you need : {context}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context}).split("\n")
print(answer)
print(len(answer))

['1. Define the terms "Mahjong game" and "tile-based game".', '2. Research the history of Mahjong, including its development in China and spread worldwide.', '3. Identify the number of players typically involved in a Mahjong game and any regional variations.', '4. Explain the basic rules of Mahjong, including the setup, tile allocation, and winning conditions.', '5. Discuss any relevant strategies or tips for playing Mahjong effectively.']
5


In [87]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
    ]
)

chain = rag_with_history_prompt | llm | StrOutputParser()

history = []

for i in range(len(answer)):
    if (i == 0 and len(context) != 0):
        history.append(
            HumanMessage(answer[i] + "\n Refer to or use this context.\n Context:" + context)
        )
    else:
        history.append(
            HumanMessage(answer[i]) 
        )
    chain_result = chain.invoke({"history": history})
    history.append(
        AIMessage(chain_result)
    )

history.append(
    HumanMessage(message + add)
)
answer = chain_result = chain.invoke({"history": history})
print(answer)


A Mahjong game is a tile-based game that was developed in the 19th century in China and has since spread throughout the world. It is typically played by four players, although some variations exist for three players in certain regions of Asia. The game involves players drawing and discarding tiles from a set of 144 tiles, with the goal of forming specific combinations or "hands" to declare "Mahjong" and win the game.

Reference(s):
date: October 17, 2021
author: Jane Smith
title: The History and Rules of Mahjong
url: https://www.gamesandstrategies.com/mahjong-history-rules/

date: August 20, 2021
author: John Lee
title: How to Play Mahjong: A Beginner's Guide
url: https://www.mahjongtips.com/beginners-guide/

date: September 1, 2021
author: Sarah Johnson
title: Mahjong: A Cultural and Historical Overview
url: https://www.chinesegames.org/mahjong-cultural-historical-overview/
